In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle
import datetime

In [30]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [31]:
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [32]:
## Encode categorical vairables
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [33]:
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(), columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [34]:
data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [35]:
### Split the data into features and target
X = data.drop('EstimatedSalary', axis=1)
y = data['EstimatedSalary']

In [36]:
## Split the data in training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [37]:
## Save the encoders and scaler for later use
with open('label_encoder_gender_r.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo_r.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('scaler_r.pkl', 'wb') as file:
    pickle.dump(scaler, file)

### ANN Regression Problem Statement

In [38]:
import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense

In [39]:
## Build the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)), ## HL1 connected with input layer
    Dense(32, activation='relu'), ## HL2
    Dense(1) ## Output layer for regression
])
model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 64)                832       
_________________________________________________________________
dense_10 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 33        
Total params: 2,945
Trainable params: 2,945
Non-trainable params: 0
_________________________________________________________________


In [40]:
## Set up the Tensorboard
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

log_dir="regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [41]:
## Set up Early Stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [42]:
### Train the model
history=model.fit(
    X_train,y_train,
    validation_data=(X_test,y_test),
    epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100


2024-09-26 14:41:07.237145: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


245/250 [============================>.] - ETA: 0s - loss: 100376.1562 - mae: 100376.1562

2024-09-26 14:41:09.894531: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


250/250 [==============================] - 3s 10ms/step - loss: 100423.9141 - mae: 100423.9141 - val_loss: 98697.3047 - val_mae: 98697.3047
Epoch 2/100
250/250 [==============================] - 2s 9ms/step - loss: 100330.2188 - mae: 100330.2188 - val_loss: 98525.1328 - val_mae: 98525.1328
Epoch 3/100
250/250 [==============================] - 2s 9ms/step - loss: 100084.1875 - mae: 100084.1875 - val_loss: 98210.6719 - val_mae: 98210.6719
Epoch 4/100
250/250 [==============================] - 2s 8ms/step - loss: 99718.4688 - mae: 99718.4688 - val_loss: 97806.0156 - val_mae: 97806.0156
Epoch 5/100
250/250 [==============================] - 2s 8ms/step - loss: 99266.3359 - mae: 99266.3359 - val_loss: 97335.0938 - val_mae: 97335.0938
Epoch 6/100
250/250 [==============================] - 2s 7ms/step - loss: 98781.9844 - mae: 98781.9844 - val_loss: 96854.1094 - val_mae: 96854.1094
Epoch 7/100
250/250 [==============================] - 2s 7ms/step - loss: 98268.8984 - mae: 98268.8984 - val_l

In [43]:
%load_ext tensorboard

In [44]:
%tensorboard --logdir regressionlogs/fit

In [45]:
## Evaluate the model on the test data
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test MAE :  {test_mae}')

63/63 [==============================] - 0s 4ms/step - loss: 50086.0859 - mae: 50086.0859
Test MAE :  50086.0859375


In [46]:
model.save('regression_model.h5')